In [8]:
import csv
import pandas as pd
import numpy as np

In [ ]:
import numpy as np

def estimate_imu_noise_and_covariance(df, sample_rate=30.0):
    # Đảm bảo đúng tên cột (xoá khoảng trắng nếu cần)
    gyro_data = df[[' angRateXRaw', ' angRateYRaw', ' angRateZRaw']].to_numpy()
    acc_data = df[[' accXRaw', ' accYRaw', ' accZRaw']].to_numpy()

    # Step 1: Tính NoiseGyro và NoiseAcc (white noise - tức là độ lệch chuẩn per √Hz)
    # Lấy đoạn dữ liệu IMU khi xe đứng yên
    gyro_var = np.var(gyro_data, axis=0)
    acc_var = np.var(acc_data, axis=0)

    # Chuẩn hóa noise theo √Hz (vì variance = sigma^2 * sample_rate)
    gyro_noise_std = np.sqrt(np.mean(gyro_var) * sample_rate)
    acc_noise_std = np.sqrt(np.mean(acc_var) * sample_rate)

    # Step 2: Ước lượng Bias Random Walk (GyroWalk & AccWalk)
    # Chia thành nhiều cửa sổ để tính drift của bias (moving mean)
    window_size = int(sample_rate)  # 1 giây
    num_windows = len(gyro_data) // window_size

    gyro_window_means = np.array([
        np.mean(gyro_data[i * window_size:(i + 1) * window_size], axis=0)
        for i in range(num_windows)
    ])
    acc_window_means = np.array([
        np.mean(acc_data[i * window_size:(i + 1) * window_size], axis=0)
        for i in range(num_windows)
    ])

    gyro_walk_std = np.sqrt(np.mean(np.var(gyro_window_means, axis=0)) * sample_rate)
    acc_walk_std = np.sqrt(np.mean(np.var(acc_window_means, axis=0)) * sample_rate)

    return {
        'IMU.NoiseGyro': gyro_noise_std,
        'IMU.NoiseAcc': acc_noise_std,
        'IMU.GyroWalk': gyro_walk_std,
        'IMU.AccWalk': acc_walk_std
    }


In [10]:
file_path = '/home/huynt119/Documents/Github/stereo_msckf/smsckf_phenikaa_data/data_raw/imu.csv'
df = pd.read_csv(file_path)
df = df[df['#IMUtimestamp'] <= 1743694157167]
df.shape[0]

3262

In [11]:
estimate_imu_noise_and_covariance(df, sample_rate=30)

{'IMU.NoiseGyro': 0.02425241347023027,
 'IMU.NoiseAcc': 1.2712282211102133,
 'IMU.GyroWalk': 0.0025184238812816146,
 'IMU.AccWalk': 0.1386052285689714}